## Setup

###### - 💡  **TIP:** The _"Setup"_ should always be executed.

### Import packages, utils and authenticate GEE

In [ ]:
import sys
from pathlib import Path

# Importing the utils module
repo_root = Path().resolve().parent 
scripts_dir = str(repo_root / "scripts")
sys.path.append(scripts_dir)

from setup import *

### Set the parameters

In [ ]:
# Initialize static parameters
dw_vis, class_labels, base_path, potential_sub_areas, palette = utils.initialize_static_parameters()

# Set up dropdown widgets and get the selection dictionary
park_dropdown, year_start_dropdown, year_end_dropdown, selection = utils.setup_dropdown_widgets(base_path, potential_sub_areas)

# Display widgets
display(park_dropdown, year_start_dropdown, year_end_dropdown)

## 1. Data collection

###### - 💡  **TIP:** The execution of _"1. Data collection"_ is only requited if other timeframes (=! 2016-2023) or AoIs are required, otherwise one can skip to 2. & 3. directly.

### 1.1. Fetching the LULC data in GEE (server-side) according to above set parameters

In [ ]:
# Loading the DW LULC data for the selected area and years server-side
results_per_area_and_year = utils.load_dw_results(selection, base_path, selection["Sub Areas"])

### 1.2. Creating the "Fisnet" (to avoid API overload)

###### - ⚠️ **NOTE:** This takes a bit longer to calculate but helps not to overlaod the API, so please be patient.
###### - 💡  **TIP:** If you experience errors in Step 1.3 try different h_- & v_intervals.

In [ ]:
# Load dissolved parks dataset
parks = utils.load_parks_and_buffers(base_path)

# Generate fishnet grid for study area
fishnet = utils.generate_fishnet(parks, base_path, selection, h_interval=1.0, v_interval=1.0)

# Pre-process window geometries and sub-area polygon intersections
window_geometries_per_sub_area = utils.process_window_geometries(selection, base_path, selection["Sub Areas"], fishnet)

### 1.3. Calculate the LULC and LULC change in AoI

###### - ⚠️ NOTE: This is the most time-consuming part of the script (depending on the size of the park) BUT once all the data is fetched from server side the plotting can begin and is quick.
###### - 💡  **TIP:** the csv's are saved to ...\LULC_change_buffer_zones\data\DW_datasets.

#### 1.3.1. Calculate the number of pixels per LULC class

In [ ]:
# Step 1: Export LULC classification data for the selected area and years to the local machine 
utils.save_lulc_data(selection, base_path, window_geometries_per_sub_area, results_per_area_and_year, class_labels)

#### 1.3.2. Calculate the number of LULC **change** pixels

In [ ]:
# Step 2: Compute LULC changes between years (server-side computation therefore very fast)
results_per_area_and_year_pairs = utils.compute_lulc_changes(selection, base_path, results_per_area_and_year)

# Step 3: Export LULC change data as caculated in Step 2 to the local machine 
utils.process_lulc_transitions(selection, base_path, window_geometries_per_sub_area, results_per_area_and_year_pairs, class_labels)

## 2. Data visualisation

### 2.1 Visualise the LULC on a map

#### 2.1.1. Load the data and the map

In [ ]:
# Loading the DW LULC data for the selected area and years server-side
results_per_area_and_year = utils.load_dw_results(selection, base_path, selection["Sub Areas"])

# Initialize and display the interactive map
Map = geemap.Map(center=[-30.5595, 22.9375], zoom=5.5, basemap='Esri.WorldImagery')
Map

#### 2.1.2. Add the data to the above map

###### - 💡  **TIP:** Both the _sub area-_ (Parks, PNA, CPA, VPA and Dissolved) and _years_-parameters (2016 - year before current year) can be set to any of the available options.

In [ ]:
# Displaying the data on the map
utils.plot_lulc(Map, selection, results_per_area_and_year, dw_vis, sub_area="CPA", years=[2018, 2020])

### 2.2. Line and pie graphs to characterize the LULC in the buffer zones

In [ ]:
# Plot the LULC comparison graphs
utils.plot_lulc_comparison(selection, class_labels, palette)

### 2.2. Bar graphs of the normalized yearly difference

In [ ]:
# Compute and plot normalized differences
utils.plot_normalized_differences(selection, class_labels, palette)

### 2.2. Sankey diagrams for LULC change

In [ ]:
# Step 1: Load the dissolved LULC change data
dissolved_change_df = utils.load_dissolved_change_data(selection)

# Sep 2: Process the data for the Sankey diagram
sources, targets, values, node_labels, node_colors = utils.process_sankey_data(dissolved_change_df, selection, class_labels, palette)

# Step 3: Plot the Sankey diagram
utils.plot_sankey_diagram(sources, targets, values, node_labels, node_colors, selection)

## 3. LULC change intensity analysis

###### 📖 **Adopted method from:** Aldwaik, S.Z. & Pontius, R.G., Jr. 2012. Intensity analysis to unify measurements of size and stationarity of land changes by interval, category, and transition. Landscape and urban planning, 106(1), pp.103-114. https://doi.org/10.1016/j.landurbplan.2012.02.010 

###### - 💡  **TIP:** These plots are not automatically saved and should be saved manually if needed for further reports.

### 3.1. Choose sub-area for further investigation

In [ ]:
# Now sub_areas is available globally for the dropdown
sub_area_dropdown = utils.setup_sub_area_dropdown(selection["Sub Areas"])
display(sub_area_dropdown)

### 3.2. Prepare data in "cross-tabulation matrix" format

In [ ]:
# Creates a dataframe with the transition matrices for the selected sub-area
df = utils.load_sub_area_data(selection, sub_area_dropdown.value)
transition_matrices = utils.compute_transition_matrices(df)

### 3.3. Calcualte the _"Time Intensity"_ of LULC change

#### 3.3.1. _"Time Intensity"_ calculation (St & U)

In [ ]:
# Step 1: Using the cross-tabulation matrix, compute the time instensity analysis
time_intervals, annual_rates_of_change, uniform_intensity, significant_intervals = utils.compute_time_intensity(transition_matrices)

# Step 2: Plot the results of the time intensity analysis
utils.plot_time_intensity_analysis(time_intervals, annual_rates_of_change, uniform_intensity, selection, sub_area_dropdown.value)

### 3.4. Calcualte the _"Category Intensity"_ of LULC change

#### 3.4.1. _"Category Intensity"_ calculation (Gtj & Lti)

###### - ⚠️ **NOTE:** The code was developed to only analyse those years with St > U as identified above (i.e., _"Fast"_ transitions).

In [ ]:
# Step 1: Compute category intensities for significant time intervals
category_intensities = utils.calculate_category_intensities(significant_intervals, transition_matrices)

# Step 2: Plot the category intensity analysis for each significant interval
utils.plot_category_intensity_analysis(significant_intervals, category_intensities, time_intervals, annual_rates_of_change, selection, sub_area_dropdown.value)

### 3.5. Calcualte the _"Transition Intensity"_ of LULC change (**Qtmj, Vtm, Rtin and Wtn**)

#### 3.5.1. _"Transition Intensity"_ calculation for **Qtmj & Vtm** -> Loss, transition **from target category** to all other categories.

In [ ]:
# Compute category transition intensities
category_transitions = utils.calculate_category_transition_intensities(significant_intervals, transition_matrices, duration_of_interval=1)

# Plot transition intensity heatmaps
utils.plot_transition_heatmap(significant_intervals, category_transitions, selection, sub_area_dropdown.value)

#### 3.5.3. _"Transition Intensity"_ calculation for **Rtin & Wtn** -> Gain, transition **to target category** from all other categories.

In [ ]:
# Compute category transition intensities
category_transitions_v2 = utils.calculate_category_transition_intensities_v2(significant_intervals, transition_matrices, duration_of_interval=1)

# Plot transition intensity heatmaps
utils.plot_transition_heatmap_v2(significant_intervals, category_transitions_v2, selection, sub_area_dropdown.value)

# 4. Visualise the LULC change hotspots between two years

### 4.1. Choose year interval to visualise

In [ ]:
# Setup the dropdown for selecting the time interval
pre_post_year_dropdown = utils.setup_pre_post_year_dropdown(significant_intervals)
pre_post_year_dropdown.observe(utils.on_pre_post_year_change, names='value')
display(pre_post_year_dropdown)

# Generate the selection widget and transition label map
lulcc_select_widget, transition_label_map = utils.setup_lulcc_change_selection(selection, sub_area_dropdown.value)
# Explicitly display the widget
display(lulcc_select_widget)

### 4.2. Dispaly the spatial extent of these LULC changes on the map

In [ ]:
# Loading the DW LULC data for the selected area and years server-side
results_per_area_and_year = utils.load_dw_results(selection, base_path, selection["Sub Areas"])

# Compute LULC changes between years (server-side computation therefore very fast)
results_per_area_and_year_pairs = utils.compute_lulc_changes(selection, base_path, results_per_area_and_year)

# Generate and display the filtered LULCC map using the selections
utils.generate_filtered_lulcc_map(results_per_area_and_year_pairs, sub_area_dropdown, pre_post_year_dropdown, lulcc_select_widget, transition_label_map)

In [ ]:
#--------------------------------------------------END OF CODE----------------------------------------------------#